In [62]:
# load API key
import os
from dotenv import load_dotenv
import openai

# to log
import logging

from typing import List

# parse json file with transcribed videos
import json

from llama_index import (Document, 
                         VectorStoreIndex,
                         StorageContext,
                         load_index_from_storage,
                         ServiceContext
)
from llama_index.evaluation import (DatasetGenerator, 
                                    RelevancyEvaluator, 
                                    FaithfulnessEvaluator,
                                    generate_question_context_pairs,
                                    EmbeddingQAFinetuneDataset
)

from llama_index.llms import OpenAI

from llama_index.node_parser import SimpleNodeParser

import pandas as pd



In [3]:
# make the async code working in the notebook cells
import nest_asyncio

nest_asyncio.apply()

In [14]:
# get the API key to the variable
load_dotenv(dotenv_path='../.env')

openai.api_key = os.getenv("OPENAI_API_KEY")

In [15]:
# set the logging
logging.basicConfig(filename='question_generation.log',  # Name of the log file
                    level=logging.DEBUG,     # Logging level
                    format='%(asctime)s - %(levelname)s - %(message)s',  # Format of log messages
                    filemode='w')  # Overwrite the log file on each script run

logging.debug('Run started')

some of the videos would have more than one node, and the last node may be too short to be used for the questions. We need to discard such nodes.

In [16]:
def process_nodes(nodes):
    """
    Processes a list of nodes based on the following criteria:
    
    1. If a node has the same title as the previous one, it's added to the result list 
       only if its word count is more than 50% of the previous node's word count.
    2. If a node has a different title, it's simply added to the result list.

    Parameters:
    - nodes (list): A list of BaseNode objects.

    Returns:
    - list: A processed list of BaseNode objects.
    """
    
    # Initialize an empty list to store the processed nodes
    result = []

    # Initialize variables to keep track of the previous node's title and word count
    prev_title = None
    prev_word_count = 0

    # Iterate through the list of nodes
    for node in nodes:
        current_title = node.metadata['title']
        current_word_count = len(node.text.split())

        # Check if the current node has the same title as the previous one
        if current_title == prev_title:
            if current_word_count > 0.5 * prev_word_count:
                result.append(node)
        else:
            result.append(node)

        # Update the previous title and word count for the next iteration
        prev_title = current_title
        prev_word_count = current_word_count

    return result


In [17]:
# open json with transcribed videos and parse
with open(f"../data/video_info.json", "r", encoding="utf-8") as f:
    data = json.load(f)

data = data[:3]

docs = [
    Document(
        text="".join(d["text"]),
        metadata={"url": d["url"][0], "title": d["title"][0]},
    )
    for d in data
]



In [58]:
# dump the truncated video json to a file

with open('video_info_test.json', 'w', encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [56]:
# set chunk_size and parse the docs into the nodes
chunk_size = 1024 * 3
parser = SimpleNodeParser.from_defaults(chunk_size=chunk_size)

nodes = parser.get_nodes_from_documents(docs)

In [19]:
# initialize the model
gpt3 = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context_gpt3 = ServiceContext.from_defaults(llm=gpt3, node_parser=parser)

now try with `generate_question_context_pairs` function to have node - question link preserved

In [26]:
# make a prompt template
qa_generate_prompt_tmpl = """\
Внизу указана контекстная информация.

---------------------
{context_str}
---------------------

На основе приведенного текста составь только один вопрос, \
на который можно ответить с помощью текста. \
Вопрос должен покрыть как можно больше аспектов в тексте. \
Он должен быть только на основе привденного текста \
и относиться к области анализа данных."
"""

# query based on the nodes list
qa_dataset = generate_question_context_pairs(
    nodes=nodes, 
    llm=gpt3, 
    qa_generate_prompt_tmpl=qa_generate_prompt_tmpl,
    num_questions_per_chunk=1 # this wil anyway not appear in the query, can be any value :)
    )

100%|██████████| 5/5 [00:10<00:00,  2.01s/it]


In [54]:
# show the questions generated

for q in qa_dataset.queries.values():
    print(q)

Какие задачи решает ML-симулятор и как он помогает инженерам по машинному обучению при приеме на работу и в работе?
Как Redash отображает дату и время в своих результатных данных и как они хранятся в базе данных PostgreSQL?
Как Redash отображает данные и как это может повлиять на результаты анализа данных?
Какие нюансы и проблемы могут возникнуть при работе с данными в Redash, связанные с форматированием чисел и дат?
Как можно использовать обратный эксперимент и сравнение графиков для анализа влияния ковида на показы рекламы мобильных игр?


In [55]:
# show the question - node pairs

for key, value in qa_dataset.dict().items():
    print(key, ':', value)

queries : {'80ffe444-7500-42a3-9bb3-5580dd745125': 'Какие задачи решает ML-симулятор и как он помогает инженерам по машинному обучению при приеме на работу и в работе?', '1102533d-17ad-43ce-8b1b-04fa8b76c1e8': 'Как Redash отображает дату и время в своих результатных данных и как они хранятся в базе данных PostgreSQL?', 'a60f0d8d-4dd3-4dc2-a233-2b5c9223fe1a': 'Как Redash отображает данные и как это может повлиять на результаты анализа данных?', '97d90959-ee63-456e-8d55-1067d8180a2a': 'Какие нюансы и проблемы могут возникнуть при работе с данными в Redash, связанные с форматированием чисел и дат?', 'c822a435-6d44-4dca-bd5d-5c05014ec15d': 'Как можно использовать обратный эксперимент и сравнение графиков для анализа влияния ковида на показы рекламы мобильных игр?'}
corpus : {'5c86129f-2559-4640-b908-c25b3da2ee60': 'Без опыта работы человека часто не берут на работу. А без работы ему часто не получается получить опыта на работе. Классический парадокс, бесконечная петля. Если вы думаете, что

In [59]:
# save the json of the pairs - have a look at it yourself!
with open('qa_dataset.json', 'w', encoding="utf-8") as f:
    json.dump(qa_dataset.dict(), f, ensure_ascii=False, indent=4)

so here we have a connection of node id - question id.  

Let's do the following operations:
1. get a connection node_id - video_url using nodes list

2. get a connection question_id - node_id using our generated q-a dataset
3. get a connection question_id - video_url using 2 previous connections
4. invert the dict 3 to have video_url: [question_ids list] pairs
5. iterate over dict 4, replace question_id by the corresponding question_text
6. iterate over original video json and for each dict, append question dict with corresponding url key

In [ ]:
for d in data:
    print(d)

# it is the way video json looks

{'url': ['https://www.youtube.com/watch?v=Api2RW4ogR4'], 'title': ['Зачем нужно проходить Симулятор ML? | karpov.courses'], 'description': ['Симулятор ML: https://bit.ly/3E4jQos\n\nНа старте карьеры многие из нас попадали в замкнутый круг «чтобы получить работу – нужно показать опыт; чтобы получить опыт – нужно, чтобы тебя взяли на работу». Как его разорвать? Найти стажировку? Сделать свой пет-проект?.. \n\nМы решили помочь вырваться из этого парадокса и создали симулятор инженера по машинному обучению. Здесь вы научитесь решать задачи, с которыми инженеры сталкиваются ежедневно. Уже в первые месяцы тренировок вы подружитесь со всеми необходимыми для работы инструментами, начнёте понимать полный цикл построения ML-систем и узнаете, как с помощью машинного обучения приносить бизнесу прибыль, что повысит вашу ценность для компании.'], 'audio_path': ['../data/audio/Api2RW4ogR4.mp4'], 'text': ['Без опыта работы человека часто не берут на работу. А без работы ему часто не получается получит

In [ ]:
nodes

# our nodes

[TextNode(id_='5c86129f-2559-4640-b908-c25b3da2ee60', embedding=None, metadata={'url': 'https://www.youtube.com/watch?v=Api2RW4ogR4', 'title': 'Зачем нужно проходить Симулятор ML? | karpov.courses'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e972e771-565e-4ee5-aa92-dabc94e8f857', node_type=None, metadata={'url': 'https://www.youtube.com/watch?v=Api2RW4ogR4', 'title': 'Зачем нужно проходить Симулятор ML? | karpov.courses'}, hash='94246086de32e9b3ec60cb60db5907d8bc7e38c15ce162e44b8b938a9039b29f')}, hash='c14d34bda85e6944f8cd02215a4fd7956cf71197785d02940eb79e0faa0f597c', text='Без опыта работы человека часто не берут на работу. А без работы ему часто не получается получить опыта на работе. Классический парадокс, бесконечная петля. Если вы думаете, что она возникает везде, но не в машинном обучении, то это не так. В подробнейшем большинстве обучающих курсов вам покажут, как обучать модели, как при

In [ ]:
# 1. pairs node id: original video url

node_connector = {node.id_: node.metadata['url'] for node in nodes}
node_connector

{'5c86129f-2559-4640-b908-c25b3da2ee60': 'https://www.youtube.com/watch?v=Api2RW4ogR4',
 '7002e54c-7bd2-4b2e-8883-9a16a1421d58': 'https://www.youtube.com/watch?v=kYcA_RVDNWM',
 '62351074-e1f7-4182-ae4a-eef577eff402': 'https://www.youtube.com/watch?v=kYcA_RVDNWM',
 '84d01643-d9c8-44f1-9249-8e10c5a9a243': 'https://www.youtube.com/watch?v=kYcA_RVDNWM',
 '549c9516-c8d9-432e-a3df-cc1fb928d7c4': 'https://www.youtube.com/watch?v=NcDcxOqB52k'}

we have connection node_id - video url. let's use it

now connect question id to its node id

In [43]:
# 2. pairs question id: node id

question_connector = {question_id: qa_dataset.relevant_docs[question_id][0] for question_id in qa_dataset.relevant_docs}
question_connector

{'80ffe444-7500-42a3-9bb3-5580dd745125': '5c86129f-2559-4640-b908-c25b3da2ee60',
 '1102533d-17ad-43ce-8b1b-04fa8b76c1e8': '7002e54c-7bd2-4b2e-8883-9a16a1421d58',
 'a60f0d8d-4dd3-4dc2-a233-2b5c9223fe1a': '62351074-e1f7-4182-ae4a-eef577eff402',
 '97d90959-ee63-456e-8d55-1067d8180a2a': '84d01643-d9c8-44f1-9249-8e10c5a9a243',
 'c822a435-6d44-4dca-bd5d-5c05014ec15d': '549c9516-c8d9-432e-a3df-cc1fb928d7c4'}

now connect question id to its video url

In [44]:
# 3. pairs question id: url

question_id_url = {question_id: node_connector[question_connector[question_id]] for question_id in question_connector}

question_id_url

{'80ffe444-7500-42a3-9bb3-5580dd745125': 'https://www.youtube.com/watch?v=Api2RW4ogR4',
 '1102533d-17ad-43ce-8b1b-04fa8b76c1e8': 'https://www.youtube.com/watch?v=kYcA_RVDNWM',
 'a60f0d8d-4dd3-4dc2-a233-2b5c9223fe1a': 'https://www.youtube.com/watch?v=kYcA_RVDNWM',
 '97d90959-ee63-456e-8d55-1067d8180a2a': 'https://www.youtube.com/watch?v=kYcA_RVDNWM',
 'c822a435-6d44-4dca-bd5d-5c05014ec15d': 'https://www.youtube.com/watch?v=NcDcxOqB52k'}

revert the pairs to have a dict of {video url: [question_ids]}

In [46]:
# 4. pairs url: question id (in a list)
url_question_id = {}

for key, value in question_id_url.items():
    if value in url_question_id:
        url_question_id[value].append(key)
    else:
        url_question_id[value] = [key]

url_question_id



{'https://www.youtube.com/watch?v=Api2RW4ogR4': ['80ffe444-7500-42a3-9bb3-5580dd745125'],
 'https://www.youtube.com/watch?v=kYcA_RVDNWM': ['1102533d-17ad-43ce-8b1b-04fa8b76c1e8',
  'a60f0d8d-4dd3-4dc2-a233-2b5c9223fe1a',
  '97d90959-ee63-456e-8d55-1067d8180a2a'],
 'https://www.youtube.com/watch?v=NcDcxOqB52k': ['c822a435-6d44-4dca-bd5d-5c05014ec15d']}

replace question ids by their text values

In [47]:
# 5. url: questions

url_question = {url: [qa_dataset.queries[question_id] for question_id in url_question_id[url]] for url in url_question_id}

url_question



{'https://www.youtube.com/watch?v=Api2RW4ogR4': ['Какие задачи решает ML-симулятор и как он помогает инженерам по машинному обучению при приеме на работу и в работе?'],
 'https://www.youtube.com/watch?v=kYcA_RVDNWM': ['Как Redash отображает дату и время в своих результатных данных и как они хранятся в базе данных PostgreSQL?',
  'Как Redash отображает данные и как это может повлиять на результаты анализа данных?',
  'Какие нюансы и проблемы могут возникнуть при работе с данными в Redash, связанные с форматированием чисел и дат?'],
 'https://www.youtube.com/watch?v=NcDcxOqB52k': ['Как можно использовать обратный эксперимент и сравнение графиков для анализа влияния ковида на показы рекламы мобильных игр?']}

modify the original video json - append question pairs to each video dict

In [50]:
# 6 extend the original json

for video in data:
    video['control_questions'] = url_question[video['url'][0]]

thats how video json looks now

In [51]:
data

[{'url': ['https://www.youtube.com/watch?v=Api2RW4ogR4'],
  'title': ['Зачем нужно проходить Симулятор ML? | karpov.courses'],
  'description': ['Симулятор ML: https://bit.ly/3E4jQos\n\nНа старте карьеры многие из нас попадали в замкнутый круг «чтобы получить работу – нужно показать опыт; чтобы получить опыт – нужно, чтобы тебя взяли на работу». Как его разорвать? Найти стажировку? Сделать свой пет-проект?.. \n\nМы решили помочь вырваться из этого парадокса и создали симулятор инженера по машинному обучению. Здесь вы научитесь решать задачи, с которыми инженеры сталкиваются ежедневно. Уже в первые месяцы тренировок вы подружитесь со всеми необходимыми для работы инструментами, начнёте понимать полный цикл построения ML-систем и узнаете, как с помощью машинного обучения приносить бизнесу прибыль, что повысит вашу ценность для компании.'],
  'audio_path': ['../data/audio/Api2RW4ogR4.mp4'],
  'text': ['Без опыта работы человека часто не берут на работу. А без работы ему часто не получаетс

In [60]:
# dump the final json

with open('video_info_test_with_questions.json', "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

now wrap it to a function `question_gen` will be used in `evalutate_utils.py`

In [8]:
from typing import List
from llama_index.schema import BaseNode

# parse json file with transcribed videos
# and save a modified file
import json

# to pass the model object into the functions
from llama_index.llms import OpenAI

# initialize the Document object to be used by SimpleNodeParser
from llama_index import Document

# to split the video text into the nodes
from llama_index.node_parser import SimpleNodeParser

# make node: question pairs
from llama_index.evaluation import generate_question_context_pairs


def process_nodes(nodes: List[BaseNode]) -> List[BaseNode]:
    """
    Processes a list of nodes based on the following criteria:
    
    1. If a node has the same title as the previous one, it's added to the result list 
       only if its word count is more than 50% of the previous node's word count.
    2. If a node has a different title, it's simply added to the result list.

    Parameters:
    - nodes (list): A list of BaseNode objects.

    Returns:
    - list: A processed list of BaseNode objects.
    """
    
    # Initialize an empty list to store the processed nodes
    result = []

    # Initialize variables to keep track of the previous node's title and word count
    prev_title = None
    prev_word_count = 0

    # Iterate through the list of nodes
    for node in nodes:
        current_title = node.metadata['title']
        current_word_count = len(node.text.split())

        # Check if the current node has the same title as the previous one
        if current_title == prev_title:
            if current_word_count > 0.5 * prev_word_count:
                result.append(node)
        else:
            result.append(node)

        # Update the previous title and word count for the next iteration
        prev_title = current_title
        prev_word_count = current_word_count

    return result

def generate_video_questions(
        video_info_path: str,
        llm: OpenAI,
        chunk_size: int = 3 * 1024,
        video_info_output_path: str = None,
        test_version: bool = True
        ) -> None:
    """
    Генерирует вопросы к тексту каждого видео из video_info.json, разбивая текст предварительно
    на 'chunk_size' токенов. Сохраняет сгенерированные вопросы в video_info.json.
    Шаблон получившегося video_info.json:
    [
        {
            "url": [<url of video>],
            "title": [<title of video>],
            "description": [<description of video>],
            "audio_path": [<path to audio track of video>],
            "text": [<text of video>],
            "control_questions": [<generated questions>]
        },
    ]

    Parameters
    ----------
    video_info_path: str
        path to video_info.json

    Returns
    -------

    """

    # create an output json path
    if not video_info_output_path:
        video_info_output_path = video_info_path.replace('.json', '_with_questions.json')

    # open json with transcribed videos and parse
    with open(video_info_path, "r", encoding="utf-8") as f:
        video_json = json.load(f)

    # when testing, do not run the function on thr whole json dataset
    if test_version:
        video_json = video_json[:3]

    assert len(video_json) == 3

    # a list of Documents to be used by SimpleNodeParser
    # to make a list of Nodes
    docs = [
        Document(
            text="".join(video_dict["text"]),
            metadata={"url": video_dict["url"][0], "title": video_dict["title"][0]},
        )
        for video_dict in video_json
    ]

    # parse the docs into the nodes
    parser = SimpleNodeParser.from_defaults(chunk_size=chunk_size)
    nodes = parser.get_nodes_from_documents(docs)

    # process the nodes to discard the last node of the video 
    # if it is very shortened during splitting
    nodes = process_nodes(nodes)

    # make a prompt template
    qa_generate_prompt_tmpl = """\
    Внизу указана контекстная информация.

    ---------------------
    {context_str}
    ---------------------

    На основе приведенного текста составь только один вопрос, \
    на который можно ответить с помощью текста. \
    Вопрос должен покрыть как можно больше аспектов в тексте. \
    Он должен быть только на основе привденного текста \
    и относиться к области анализа данных."
    """

    # query based on the nodes list
    # num_questions_per_chunk will anyway not appear in the query, 
    # can be any value :)
    qa_dataset = generate_question_context_pairs(
        nodes=nodes, 
        llm=llm, 
        qa_generate_prompt_tmpl=qa_generate_prompt_tmpl,
        num_questions_per_chunk=1 
        )

    # pairs node_id: video_url
    node_id_url = {node.id_: node.metadata['url'] for node in nodes}

    # pairs question_id: node_id
    question_id_node_id = {
        question_id: qa_dataset.relevant_docs[question_id][0] 
        for question_id in qa_dataset.relevant_docs
        }
    
    # pairs question_id: video_url
    question_id_url = {
        question_id: node_id_url[question_id_node_id[question_id]] 
        for question_id in question_id_node_id
        }
    
    # invert the 'question_id_url'
    # pairs video_url: list[question_id]
    url_question_id = {}

    for key, value in question_id_url.items():
        if value in url_question_id:
            url_question_id[value].append(key)
        else:
            url_question_id[value] = [key]
    
    # replace question_id by question_text
    # pairs video_url: list[question]

    url_question = {
        url: [
            qa_dataset.queries[question_id] 
              for question_id in url_question_id[url]
              ] 
              for url in url_question_id
              }
    
    # extend the original json
    # by creating 'control_questions': list[question] pairs
    # for each video's dict
    for video_dict in video_json:
        video_dict['control_questions'] = url_question[video_dict['url'][0]]
    
    # dump the final json to a new file
    with open(video_info_output_path, "w", encoding="utf-8") as f:
        json.dump(video_json, f, ensure_ascii=False, indent=4)


In [9]:
# make the async code working in the notebook cells
import nest_asyncio
from dotenv import load_dotenv
import os
import openai

nest_asyncio.apply()

# get the API key to the variable
load_dotenv(dotenv_path='../.env')

openai.api_key = os.getenv("OPENAI_API_KEY")

# initialize the model
gpt3 = OpenAI(temperature=0, model="gpt-3.5-turbo")

generate_video_questions(
        video_info_path='../data/video_info.json',
        llm=gpt3,
        chunk_size = 3 * 1024,
        test_version=True
        )

100%|██████████| 5/5 [00:12<00:00,  2.41s/it]


Вариант 1 - сносно

In [138]:
question_gen_query = "Ты - начинающий спецалист в анализе данных. \
                        На основе приведенного текста составь только один вопрос, \
                        на который можно ответить с помощью текста. \
                        Вопрос должен покрыть как можно больше аспектов в тексте. \
                        Он должен быть только на основе привденного текста \
                        и относиться к области анализа данных" 

data_generator_nodes = DatasetGenerator(
    nodes=nodes[:2],
    service_context=service_context_gpt3,
    question_gen_query=question_gen_query
    )

eval_questions_nodes = data_generator_nodes.generate_questions_from_nodes()

eval_questions_nodes


['Какие задачи можно решить с помощью ML-симулятора?',
 'Какие данные хранятся в базе данных PostgreSQL?']

Вариант 2 - лучший

In [143]:
question_gen_query = "На основе приведенного текста составь только один вопрос, \
                        на который можно ответить с помощью текста. \
                        Вопрос должен покрыть как можно больше аспектов в тексте. \
                        Он должен быть только на основе привденного текста \
                        и относиться к области анализа данных" 

data_generator_nodes = DatasetGenerator(
    nodes=nodes[:2],
    service_context=service_context_gpt3,
    question_gen_query=question_gen_query
    )

eval_questions_nodes = data_generator_nodes.generate_questions_from_nodes()

eval_questions_nodes


['Какие задачи решает ML-симулятор и как он помогает инженеру по машинному обучению?',
 'Какие важные нюансы нужно учитывать при хранении данных в Redash и как это может повлиять на аналитический анализ данных?']

Вариант 3 - дает больше одного

In [140]:
question_gen_query = "Я хочу создать список эталонных вопросов к корпусу текстов. \
                      Эталонный вопрос относится к области анализа данных\
                      и на него можно ответить с помощью соответствующего текста.\
                      Вопрос должен покрыть как можно больше аспектов в тексте.\
                      Составь один эталонный вопрос к этому тексту."


data_generator_nodes = DatasetGenerator(
    nodes=nodes[:2],
    service_context=service_context_gpt3,
    question_gen_query=question_gen_query
    )

eval_questions_nodes = data_generator_nodes.generate_questions_from_nodes()

eval_questions_nodes


['Какие задачи решает ML-симулятор?',
 'Какие данные хранятся в специальной базе данных в PostgreSQL?',
 'Как Redash отображает результаты запросов?',
 'Как Redash отображает время в своем формате?',
 'Какие форматы отображения даты доступны в Redash?',
 'Какие нюансы связаны с отображением времени в Redash?',
 'Какие проблемы могут возникнуть при отборе записей по времени в Redash?',
 'Как данные хранятся в базе данных PostgreSQL?',
 'Какой формат времени используется в базе данных PostgreSQL?',
 'Какие форматы времени поддерживает Redash?',
 'Какие различия между форматом хранения данных и их отображением в Redash?']

Вариант 4 - сносно

In [141]:
question_gen_query = "На русском языке. только один вопрос на текст! Это должен быть именно один вопрос"

data_generator_nodes = DatasetGenerator(
    nodes=nodes[:2],
    service_context=service_context_gpt3,
    question_gen_query=question_gen_query
    )

eval_questions_nodes = data_generator_nodes.generate_questions_from_nodes()

eval_questions_nodes


['Какие задачи можно решить с помощью ML-симулятора?',
 'Какие данные хранятся в базе данных PostgreSQL?']

Вариант 5 - твой вариант - больше двух вопросов на ноду

In [142]:
data_generator_nodes = DatasetGenerator(
    nodes=nodes[:2],
    service_context=service_context_gpt3,
    num_questions_per_chunk=2,
    question_gen_query="На русском языке"
    )

eval_questions_nodes = data_generator_nodes.generate_questions_from_nodes()

eval_questions_nodes


['Что такое Симулятор ML и для чего он нужен?',
 'Какие задачи можно решить с помощью Симулятора ML?',
 'Какой опыт можно получить, проходя Симулятор ML?',
 'Как Симулятор ML поможет при приеме на работу?',
 'Как Симулятор ML поможет при выполнении новых задач на работе?',
 'Что такое Redash и как он отображает данные?',
 'Где хранятся данные в Redash?',
 'Как Redash отображает время?',
 'Какой формат отображения даты использует Redash?',
 'Какие нюансы есть в отображении данных в Redash?',
 'Какие настройки есть в Redash для отображения даты?',
 'Какие форматы даты можно использовать в Redash?',
 'Какие данные хранятся в базе данных в формате PostgreSQL?',
 'Какие проблемы могут возникнуть при отборе записей по времени в Redash?',
 'Какие инструменты существуют для визуализации данных, похожие на Redash?']

Вариант 6 - пример того, как на некоторые ноды бот не приводит вопрос (см вторую ноду)

In [137]:
data_generator_nodes_proc = DatasetGenerator(
    nodes=processed_nodes,
    service_context=service_context_gpt3,
    # num_questions_per_chunk=1,
    question_gen_query=question_gen_query
    )

eval_questions_nodes_proc = data_generator_nodes_proc.generate_questions_from_nodes()

eval_questions_nodes_proc

['Какие задачи можно решить с помощью ML-симулятора?',
 'Если бы мы просто перевели формат колонки Creation Time в текстовый формат, мы могли бы увидеть, как она выглядит. Однако, в Redash текстовый формат уже не форматирует данные, поэтому мы не можем узнать, как они хранятся в базе данных PostgreSQL.',
 'Как можно интерпретировать график и сделать статистический вывод?',
 'Какие задачи решает аналитик данных в банках?',
 'Какие факторы могут влиять на то, что товарный запас не был распродан в некоторых магазинах?',
 'Какие детали и нюансы вы заметили при рассмотрении задач других аналитиков?',
 'Какие проблемы с организацией работы и атмосферой были упомянуты девочкой, которая работала в магните?',
 'Какие проблемы возникли при выполнении проектов и как вы планируете к ним подходить в будущем?']